# Train a model

We need to save our W&B credentials on the folder that will be copied to the SM instance

In [ ]:
!pip install -Uqqq transformers accelerate "datasets[s3]==2.13.0" sagemaker wandb

save your wandb API token to the script folder

In [ ]:
import wandb
wandb.sagemaker_auth(path="scripts")

## SageMaker auth

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

DATASET_S3 = f's3://{sess.default_bucket()}/processed/wandbot/train'

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

## Loading the data from W&B

In [ ]:
# the dataset we pre-tokenized and created before
AT_ADDRESS = "capecape/aws_llm_workshop/wandbot_dataset_tokenized:v0"

We put inside the `run_clm.py` file the loading from a W&B Artifact:

```python
import wandb
from datasets import load_from_disk

def load_from_artifact(at_address, at_type="dataset"):
    "Load a HF dataset from a W&B artifact"
    artifact = wandb.use_artifact(at_address, type=at_type)
    artifact_dir = artifact.download()
    return load_from_disk(artifact_dir)

...
    ds = load_from_artifact(AT_ADDRESS)

```

## Train time! 🚂

In [ ]:
WANDB_PROJECT = "aws_llm_workshop"

In [ ]:
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder

# define Training Job Name 
MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"
job_name = 'wandb-qlora-codellama7'

lr = 2e-4

# hyperparameters, which are passed into the training job
hyperparameters ={
    'model_id': MODEL_NAME,                           # pre-trained model
    # 'dataset_artifact': AT_ADDRESS,                   # Artifact containing the dataset at W&B
    # 'dataset_path': '/opt/ml/input/data/training',    # path where sagemaker will save training dataset
    'dataset_path': AT_ADDRESS,
    'epochs': 1,                                      # number of training epochs
    'per_device_train_batch_size': 2,                 # batch size for training
    'lr': lr,                                         # learning rate used during training
    'hf_token': HfFolder.get_token(),                 # huggingface token to access llama 2
    'merge_weights': True,                            # wether to merge LoRA into the model (needs more memory)
    'report_to': "wandb",                              # report to wandb
    'wandb_project': WANDB_PROJECT,
    "run_name":  f"{MODEL_NAME}__qlora",
}
    
# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_clm.py',      # train script
    source_dir           = 'scripts',         # directory which includes all the files needed for training
    instance_type        = 'ml.g5.4xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.28',            # the transformers version used in the training job
    pytorch_version      = '2.0',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache"}, # set env variable to cache models in /tmp
)

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': DATASET_S3}

# data = {}
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)